In [1]:
pwd

In [2]:
from DiamondScrapper.Scrapper import DriverScrapper

In [3]:
test = DriverScrapper(url="https://www.bluenile.com/diamond-search")

In [5]:
df = test.get_by_filter()

In [6]:
df

,Shape,Price,Carat,Cut,Color,Clarity,Polish,Symmetry,Fluorescence,Depth,Table,L/W,Price/Ct,Culet,Stock No.,Delivery Date
0,Round,"$1,379",0.80,Very Good,J,SI2,Very Good,Very Good,Medium,63.0,62.0,1.01,"$1,724",None,LD13441674,May 7
1,Round,"$1,385",0.80,Good,K,SI1,Excellent,Very Good,None,64.5,55.0,1.01,"$1,731",None,LD13083608,Apr 13
2,Round,"$1,386",0.80,Very Good,J,SI2,Excellent,Excellent,Faint,64.4,57.0,1.01,"$1,732",None,LD13554587,May 7
3,Round,"$1,390",0.80,Good,K,SI1,Excellent,Good,None,66.7,54.0,1.02,"$1,738",None,LD13163941,May 12
4,Round,"$1,433",0.80,Very Good,K,SI1,Excellent,Very Good,Strong,64.7,58.0,1.01,"$1,791",None,LD13530328,May 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,Round,"$2,176",0.82,Ideal,J,VS2,Excellent,Excellent,Strong,60.8,60.0,1.01,"$2,654",None,LD13595361,May 7
546,Round,"$2,176",0.80,Very Good,J,VS2,Excellent,Very Good,None,63.9,57.0,1.01,"$2,720",None,LD13598704,May 12
547,Round,"$2,176",0.90,Very Good,J,SI2,Excellent,Excellent,Faint,64.3,57.0,1.01,"$2,418",None,LD13639368,Apr 13
548,Round,"$2,176",0.80,Very Good,G,SI2,Excellent,Very Good,None,64.7,56.0,1.01,"$2,720",None,LD13725964,May 7


In [129]:
import requests
import urllib
from bs4 import BeautifulSoup
import pandas as pd


In [130]:
headers = {"Referer": "https://www.bluenile.com/diamonds/round-cut",
           "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36",
           "Host": "www.bluenile.com",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",}   

url = "https://www.bluenile.com/diamonds/round-cut"

## Reference links

- similar problem: https://stackoverflow.com/questions/53015772/how-to-grab-more-data
- scrapping with scrolling down: https://stackoverflow.com/questions/21006940/how-to-load-all-entries-in-an-infinite-scroll-at-once-to-parse-the-html-in-pytho
- locaating html elements: https://www.techbeamers.com/locate-elements-selenium-python/, https://selenium-python.readthedocs.io/locating-elements.html
- send input element keys: https://stackoverflow.com/questions/1629053/typing-enter-return-key-in-selenium
- ChromeDriver: https://chromedriver.chromium.org/


# Poor scrapper
only scraps 60 rows with default setting

In [131]:
s = requests.session()
s = BeautifulSoup(s.get(url,headers = headers).content, "html.parser")

headerinctags = s.find_all('div', class_='grid-header normal-header')
header = headerinctags[0].get_text(';').split(";")

diamondsmessy = s.find_all('a', class_='grid-row row')
diamondscleaned = diamondsmessy[1].get_text(";")

del header[0]
diamonds = pd.DataFrame(columns=header)

for i in range(len(diamondsmessy)):
    a = diamondsmessy[i].get_text(";")
    b = a.split(";")
    del b[4]
    a = pd.DataFrame(b, index=header)
    b = a.transpose()
    diamonds = pd.concat([diamonds, b], ignore_index=True)

In [132]:
diamonds.shape

(60, 16)

# Naive scrapper
scraps 1000 rows with scrolling down and default setting

In [102]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome('/Users/chichengzhang/log/chromedriver')

browser.get("https://www.bluenile.com/diamonds/round-cut")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 50

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns-=1
    
soup = BeautifulSoup(browser.page_source, "html.parser")

headerinctags = s.find_all('div', class_='grid-header normal-header')
header = headerinctags[0].get_text(';').split(";")

del header[0]
diamonds = pd.DataFrame(columns=header)

diamondsmessy = soup.find_all('a', class_='grid-row row')

for i in range(len(diamondsmessy)):
    a = diamondsmessy[i].get_text(";")
    b = a.split(";")
    del b[4]
    a = pd.DataFrame(b, index=header)
    b = a.transpose()
    diamonds = pd.concat([diamonds, b], ignore_index=True)
    
browser.quit()

In [101]:
diamonds.shape

(1000, 16)

In [134]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

#for fun, let's time this
start = time.time()

"""Define important numbers"""

scroll_pauze_time = 0.2 #delay after scroll
scroll_number = 30 #number of times scrolled per page
pages_visited = 25 #number of times the price is increased

# User Input Conditions
min_carat_input = 0.8
max_carat_input = 1.0

min_price_input = 1000
max_price_input = 5000

#TODO: More to be defined


"""Set up the website"""

url = "https://www.bluenile.com/diamond-search"

url_response = webdriver.Chrome("/Users/chichengzhang/log/chromedriver")
url_response.get(url)


"""Set up carat filter"""

min_carat = url_response.find_element_by_id("minValueCarat")
min_carat.click()
min_carat.send_keys("{}".format(min_carat_input))
min_carat.send_keys(Keys.ENTER)

max_carat = url_response.find_element_by_id("maxValueCarat")
max_carat.click()
max_carat.send_keys("{}".format(max_carat_input))
max_carat.send_keys(Keys.ENTER)


"""Set up price filter"""

# Cut up price range
price_cuts = np.split(np.arange(min_price_input, max_price_input), pages_visited)
min_price_cuts = np.min(price_cuts, 1)
max_price_cuts = np.max(price_cuts, 1)


"""Build DataFrame"""

soup = BeautifulSoup(url_response.page_source, "html.parser")

headerinctags = soup.find_all('div', class_='grid-header normal-header')
header = headerinctags[0].get_text(';').split(";")
del header[0]
diamonds = pd.DataFrame(columns=header)


"""Start page loop"""

for p in range(pages_visited):
    print("page number:", p)
    url_response.execute_script("window.scrollTo(0, 0)")
    
    # Reset the price filter
    min_price = url_response.find_element_by_id('minValuePrice')
    min_price.click()
    min_price.send_keys("{}".format(min_price_cuts[p]))
    min_price.send_keys(Keys.ENTER)

    max_price = url_response.find_element_by_id('maxValuePrice')
    max_price.click()
    max_price.send_keys("{}".format(max_price_cuts[p]))
    max_price.send_keys(Keys.ENTER)

    """Scroll down"""

    elem = url_response.find_element_by_tag_name("body")
    for scr in range(scroll_number):
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(scroll_pauze_time)
    
    """Scrape data"""

    s = BeautifulSoup(url_response.page_source, "html.parser")
    diamondsmessy = s.find_all('a', class_='grid-row row')

    for i in range(len(diamondsmessy)):
        a = diamondsmessy[i].get_text(";")
        b = a.split(";")
        del b[4]
        a = pd.DataFrame(b, index=header)
        b = a.transpose()
        diamonds = pd.concat([diamonds, b], ignore_index=True)


end = time.time()
print("This took "+ str(end-start)+" seconds")

page number: 0
page number: 1
page number: 2
page number: 3
page number: 4
page number: 5
page number: 6
page number: 7
page number: 8
page number: 9
page number: 10
page number: 11
page number: 12
page number: 13
page number: 14
page number: 15
page number: 16
page number: 17
page number: 18
page number: 19
page number: 20
page number: 21
page number: 22
page number: 23
page number: 24
This took 299.3553569316864 seconds


In [135]:
diamonds

,Shape,Price,Carat,Cut,Color,Clarity,Polish,Symmetry,Fluorescence,Depth,Table,L/W,Price/Ct,Culet,Stock No.,Delivery Date
0,Round,"$1,260",0.80,Good,K,SI1,Excellent,Good,Medium,67.0,65.0,1.01,"$1,575",None,LD13034307,Nov 22
1,Round,"$1,398",0.80,Very Good,K,SI2,Excellent,Very Good,Faint,64.4,54.0,1.01,"$1,748",None,LD12741355,Nov 22
2,Round,"$1,428",0.80,Good,K,SI1,Excellent,Very Good,None,64.5,55.0,1.01,"$1,785",None,LD13083608,Nov 21
3,Round,"$1,496",0.80,Ideal,K,SI2,Excellent,Very Good,Faint,61.8,58.0,1.01,"$1,870",None,LD13011970,Nov 22
4,Round,"$1,498",0.80,Ideal,K,SI2,Excellent,Excellent,None,62.8,56.0,1.01,"$1,872",None,LD12966843,Nov 21
5,Round,"$1,509",0.81,Very Good,K,SI2,Excellent,Very Good,Strong,59.7,60.0,1.01,"$1,863",None,LD12842099,Nov 22
6,Round,"$1,526",0.80,Very Good,K,SI2,Excellent,Very Good,Faint,63.6,59.0,1.01,"$1,908",None,LD12547120,Nov 20
7,Round,"$1,562",0.80,Very Good,K,SI1,Very Good,Very Good,Strong,64.3,57.0,1.01,"$1,952",None,LD13100158,Nov 21
8,Round,"$1,582",0.80,Good,K,SI1,Very Good,Very Good,None,66.4,56.0,1.01,"$1,978",None,LD12740402,Nov 22
9,Round,"$1,594",0.80,Very Good,K,SI1,Excellent,Very Good,None,63.9,57.0,1.01,"$1,992",None,LD11924660,Nov 22
